In [2]:
import os
from glob import glob
import json
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import Image, display
import time
import numpy as np
import cv2
from collections import defaultdict

In [3]:
scatter = pd.read_csv('/Users/minsukchang/Research/ChartQA/scatter/scatters.csv')


def real_corr(filename):
    direction, _, corr, exp = filename.split('_')
    row = scatter[(scatter['scatter_type'] == direction) & (scatter['corr'] == int(corr)) & (scatter['exp'] == int(exp))]
    if row.shape[0] == 0:
        print(filename)
    return row['corr_computed'].values[0]

In [4]:
labels = [
    ['As the usage of internet increases, so does the homicide rate in the city.', 'Increasing internet usage', 'Higher homicide rate in the city'],
    ['People who eat more cheese, tend to be better at dancing.', 'Eating more cheese', 'Better at dancing'],
    ['The more often students eat breakfast, the higher their GPAs are.', 'Eating breakfast more often', 'Having a higher GPA'],
    ['The more students wear glasses, the later the gym closes on campus.', 'More students wearing glasses', 'Gym closing later on campus'],
    ['A city with more lawyers, tends to have more trees.', 'Having more lawyers in a city', 'Having more trees in the city'],
    ['A worker with a longer commute, tends to be more stressed.', 'Commuting for a longer time', 'Being more stressed'],
    ['People who sleep more, tend to be happier with their lives.', 'Sleeping more', 'Being happier with life'],
    ['As the number of environmental regulations increases, so does the air quality in the city.', 'Having more environmental regulations', 'Improving air quality in the city']
]

In [5]:
def generate_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "bias/assets/bias.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "Recorded Answer:",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "parameters": {
                "radius": 100,
                "blur_std": 13,
            },
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        }
    }

In [6]:
def create_default_components():
    return {
        "welcome": {
            "type": "markdown",
            "path": "phase1/assets/welcome.md",
            "response": []
        },
        "consent": {
            "type": "markdown",
            "path": "phase1/assets/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "main_start": {
            "type": "markdown",
            "path": "phase1/assets/main_start.md",
            "response": []
        },
        "instruction": {
            "type": "markdown",
            "path": "phase1/assets/instruction.md",
            "response": []
        },
        "demographics": {
            "type": "markdown",
            "path": "phase1/assets/demographics.md",
            "response": [
                {
                    "id": "age",
                    "prompt": "What is your age?",
                    "required": True,
                    "location": "belowStimulus",
                    "type": "dropdown",
                    "options": [
                        "18~24",
                        "25~34",
                        "35~44",
                        "45~54",
                        "55~",
                        "Prefer not to say",
                    ]
                },
                {
                    "id": "gender",
                    "prompt": "What is your gender?",
                    "required": True,
                    "location": "belowStimulus",
                    "type": "dropdown",
                    "options": [
                        "Male",
                        "Female",
                        "Other",
                        "Prefer not to say"
                    ]
                }
            ]
        }
    }


def create_example_components():
    return {
        "bubble_example1": {
            "baseComponent": "bubble",
            "parameters": {
                "image": "https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/pos_scatter_2_1.png",
                "example": True,
                "correlation": real_corr("pos_scatter_2_1"),
                "label": labels[0][0],
                "X": labels[0][1],
                "Y": labels[0][2],
            }
        },
        
    }


def create_main_components():
    components = defaultdict(dict)
    for corr in [3, 5]:
        for i in range(6, 8): # for phase 1
            for direction in ["pos"]:
                for label_idx, label in enumerate(labels):
                    label, x, y = label
                    components[label][f"bubble_{corr}_{i}_{direction}_{label_idx}"] = {
                        "baseComponent": "bubble",
                        "parameters": {
                            "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                            "example": False,
                            "correlation": real_corr(f"{direction}_scatter_{corr}_{i}"),
                            "label": label,
                            "X": x,
                            "Y": y,
                            "corr": corr,
                            "exp": i,
                            "direction": direction
                        }
                    }
    return components


def sequence_generator(main_components, example_components):
    groups = [{
        "id": label[:30],
        "order": "latinSquare",
        "components": list(main_components[label].keys())
    } for label in main_components.keys()]
    sequence = {
        "order": "fixed",
        "components": [
            "welcome",
            "consent",
            "demographics",
            "instruction",
            *example_components,
            "main_start",
            {
                "id": "images",
                "order": "latinSquare",
                "components": groups
            }
        ]
    }
    return sequence

In [7]:
default_components = create_default_components()
main_components = create_main_components()
example_components = create_example_components()
components = default_components | example_components
for component in main_components.values():
    components |= component
sequence = sequence_generator(main_components, example_components)
baseComponents = generate_base_components()
print(f"Total number of components: {len(components)}")

Total number of components: 38


In [8]:
with open("config.json", "r") as f:
    config = json.load(f)
config['baseComponents'] = baseComponents
config['components'] = components
config['sequence'] = sequence

with open("config.json", "w") as f:
    json.dump(config, f, indent=4)